In [1]:
!pip install langchain==0.0.312
# !pip install --upgrade langchain
# !pip install --upgrade pip

  Using cached langchain-0.0.312-py3-none-any.whl.metadata (15 kB)
Using cached langchain-0.0.312-py3-none-any.whl (1.8 MB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.325
    Uninstalling langchain-0.0.325:
      Successfully uninstalled langchain-0.0.325


# Tools and Routing

In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
from langchain.agents import tool

In [4]:
@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

In [5]:
search.name

'search'

In [6]:
search.description

'search(query: str) -> str - Search for weather online'

In [7]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [8]:
from pydantic import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")

In [9]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [10]:
search.args

{'query': {'title': 'Query',
  'description': 'Thing to search for',
  'type': 'string'}}

In [11]:
search.run("sf")

'42f'

In [12]:
search("sf")

'42f'

In [13]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [14]:
get_current_temperature.name

'get_current_temperature'

In [15]:
get_current_temperature.description

'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.'

In [16]:
get_current_temperature.args

{'latitude': {'title': 'Latitude',
  'description': 'Latitude of the location to fetch weather data for',
  'type': 'number'},
 'longitude': {'title': 'Longitude',
  'description': 'Longitude of the location to fetch weather data for',
  'type': 'number'}}

In [17]:
from langchain.tools.render import format_tool_to_openai_function

In [18]:
format_tool_to_openai_function(get_current_temperature)

{'name': 'get_current_temperature',
 'description': 'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.',
 'parameters': {'title': 'OpenMeteoInput',
  'type': 'object',
  'properties': {'latitude': {'title': 'Latitude',
    'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'title': 'Longitude',
    'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude']}}

In [19]:
get_current_temperature({"latitude": 13, "longitude": 14})

'The current temperature is 24.7°C'

In [20]:
get_current_temperature({"latitude": 13, "longitude": 45})

'The current temperature is 30.3°C'

In [21]:
get_current_temperature({"latitude": 63, "longitude": 14})

'The current temperature is -4.9°C'

In [22]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    print("search_wikipedia", query)
    wikipedia.set_lang("ko")
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [23]:
search_wikipedia.name

'search_wikipedia'

In [24]:
search_wikipedia.description

'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.'

In [25]:
format_tool_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.',
 'parameters': {'title': 'search_wikipediaSchemaSchema',
  'type': 'object',
  'properties': {'query': {'title': 'Query', 'type': 'string'}},
  'required': ['query']}}

In [26]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec

In [27]:
text = """
{
  "openapi": "3.0.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [28]:
spec = OpenAPISpec.from_text(text)

Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


In [29]:
import langchain
langchain.__version__

'0.0.312'

In [30]:
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)

In [31]:
pet_openai_functions

[{'name': 'listPets',
  'description': 'List all pets',
  'parameters': {'type': 'object',
   'properties': {'params': {'type': 'object',
     'properties': {'limit': {'type': 'integer',
       'maximum': 100.0,
       'schema_format': 'int32',
       'description': 'How many items to return at one time (max 100)'}},
     'required': []}}}},
 {'name': 'createPets',
  'description': 'Create a pet',
  'parameters': {'type': 'object', 'properties': {}}},
 {'name': 'showPetById',
  'description': 'Info for a specific pet',
  'parameters': {'type': 'object',
   'properties': {'path_params': {'type': 'object',
     'properties': {'petId': {'type': 'string',
       'description': 'The id of the pet to retrieve'}},
     'required': ['petId']}}}}]

In [32]:
from langchain.chat_models import ChatOpenAI

In [33]:
model = ChatOpenAI(temperature=0).bind(functions=pet_openai_functions)

In [34]:
model.invoke("what are three pets names")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'listPets', 'arguments': '{\n  "params": {\n    "limit": 3\n  }\n}'}})

In [35]:
model.invoke("tell me about pet with id 42")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'showPetById', 'arguments': '{\n  "path_params": {\n    "petId": "42"\n  }\n}'}})

### Routing

In lesson 3, we show an example of function calling deciding between two candidate functions.

Given our tools above, let's format these as OpenAI functions and show this same behavior.

In [36]:
functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [37]:
functions

[{'name': 'search_wikipedia',
  'description': 'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.',
  'parameters': {'title': 'search_wikipediaSchemaSchema',
   'type': 'object',
   'properties': {'query': {'title': 'Query', 'type': 'string'}},
   'required': ['query']}},
 {'name': 'get_current_temperature',
  'description': 'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.',
  'parameters': {'title': 'OpenMeteoInput',
   'type': 'object',
   'properties': {'latitude': {'title': 'Latitude',
     'description': 'Latitude of the location to fetch weather data for',
     'type': 'number'},
    'longitude': {'title': 'Longitude',
     'description': 'Longitude of the location to fetch weather data for',
     'type': 'number'}},
   'required': ['latitude', 'longitude']}}]

In [38]:
model.invoke("what is the weather in sf right now")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{\n  "latitude": 37.7749,\n  "longitude": -122.4194\n}'}})

In [39]:
model.invoke("what is langchain")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_wikipedia', 'arguments': '{\n  "query": "langchain"\n}'}})

In [40]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model

In [41]:
chain.invoke({"input": "what is the weather in seoul right now"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{\n  "latitude": 37.5665,\n  "longitude": 126.9780\n}'}})

In [42]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [43]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [44]:
result = chain.invoke({"input": "what is the weather in seoul right now"})

In [45]:
type(result)

langchain.schema.agent.AgentActionMessageLog

In [46]:
result.tool

'get_current_temperature'

In [47]:
result.tool_input

{'latitude': 37.5665, 'longitude': 126.978}

In [48]:
result.return_values

AttributeError: 'AgentActionMessageLog' object has no attribute 'return_values'

In [49]:
get_current_temperature(result.tool_input)

'The current temperature is 18.4°C'

In [50]:
result = chain.invoke({"input": "hi!"})

In [51]:
type(result)

langchain.schema.agent.AgentFinish

In [52]:
result.return_values

{'output': 'Hello! How can I assist you today?'}

In [53]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [54]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [55]:
chain1 = prompt | model

In [56]:
chain2 = prompt | model | OpenAIFunctionsAgentOutputParser()

In [57]:
result = chain.invoke({"input": "What is the weather in san francisco right now?"})

In [58]:
chain1.invoke({"input": "What is the weather in san francisco right now?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{\n  "latitude": 37.7749,\n  "longitude": -122.4194\n}'}})

In [59]:
chain2.invoke({"input": "What is the weather in san francisco right now?"})

AgentActionMessageLog(tool='get_current_temperature', tool_input={'latitude': 37.7749, 'longitude': -122.4194}, log="\nInvoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{\n  "latitude": 37.7749,\n  "longitude": -122.4194\n}'}})])

In [60]:
result

'The current temperature is 12.6°C'

In [61]:
result = chain.invoke({"input": "서울날씨는?"})

In [62]:
result

'The current temperature is 18.4°C'

In [63]:
chain.invoke({"input": "What is langchain?"})

search_wikipedia langchain


'No good Wikipedia Search Result was found'

In [64]:
print(chain.invoke({"input": "대한민국에 대해서 설명해줘?"}))

search_wikipedia 대한민국
Page: 대한민국
Summary: 대한민국(한국 한자: 大韓民國, 영어: Republic of Korea, ROK), 약칭 한국(한국 한자: 韓國, 영어: Korea), 남한(한국 한자: 南韓, 영어: South Korea), 남조선(한국 한자: 南朝鮮)은 동아시아의 한반도 군사 분계선 남부에 위치한 국가이다. 현정체제는 대한민국 제6공화국이다. 대한민국의 국기는 대한민국 국기법에 따라 태극기이며, 국가는 관습상 애국가, 국화는 관습상 무궁화이다. 공용어는 한국어와 한국 수어이다. 수도는 서울이다. 인구는 2023년을 기준으로 약 5,174만 명으로, 전체 인구 중 절반이(약 2,624만 명) 수도권에 살고 있다.대한민국은 1948년 5월 10일 총선거를 통해 제헌국회를 구성하였고, 1948년 8월 15일 대한민국 정부를 수립하였다. 1948년 제헌 국회에서 대한민국의 국호를 계승하여 헌법에 명시하였고, 다시 1950년 1월 16일 국무원 고시 제7호 ‘국호 및 일부 지방명과 지도색 사용에 관한 건’에 의해 확정하였다.
대한민국은 20세기 후반 이후 급격한 경제 성장을 이루었다. 그 과정에서 1990년대 말 외환 위기 등의 부침이 있기도 했다. 대한민국의 2022년 1인당 국민 총소득 (GNI)은 명목 3만 4,994달러이다. 2020년 유엔개발계획 (UNDP)이 매년 발표하는 인간개발지수 (HDI) 조사에서 세계 22위를 기록하였다. 2021년 7월 2일 스위스 제네바 본부에서 열린 ‘제68차 무역개발이사회’ 마지막 회의에서 한국의 지위를 선진국 그룹으로 ‘의견 일치’로 변경하고 선진국으로 인정했다. 다만 높은 자살률, 장시간 근로 문화와 높은 산업 재해 사망률, 저출산 등의 사회 문제가 이 같은 성과와 병존하고 있다.
대한민국은 이코노미스트에서 발표하는 민주주의 지수 조사에서 2019년 기준 23위의 8.0점을 기록한 바와 같이 아시아에서 민주주의 수준이 가장 높은 국가 가운데 하나이다. 또한 대한민국은 주요 20개

In [65]:
chain.invoke({"input": "hi!"})

'Hello! How can I assist you today?'

In [66]:
print(chain.invoke({"input": "하이페리온1차에 대해서 알려줘?"}))

search_wikipedia 하이페리온1차
Page: 목동 하이페리온
Summary: 목동 현대 하이페리온(영어: Mok-dong Hyundai Hyperion) 혹은 하이페리온 타워(영어: Hyperion Tower)는 대한민국 서울특별시 양천구 목1동에 있는 초고층 마천루이자 고급 주상복합 아파트이다. 1998년에 착공하여 1차는 2003년, 2차는 2006년에 완공되었으며 1차는 3개 동, 지하 6층 ~ 지상 69층, 2차는 6개 동, 지하 4층 ~ 지상 40층으로 구성되어 있다.

Page: 삼성 타워팰리스 3차
Summary: 삼성 타워팰리스 3차(영어: Samsung Tower Palace), 삼성 타워팰리스 3차 - G동(영어: Samsung Tower Palace 3 – Tower G)는 대한민국 서울특별시 강남구 도곡동 도곡역 앞에 위치한 초고층 주상복합 마천루 단지이다. 설계는 미국의 건설업체 스키드모어, 오윙스 앤드 메릴이 맡았으며 개발은 삼성물산이 개발했고 610세대의 초고층 아파트로 구성되어 있다. 삼성 타워팰리스 단지의 A ~ G동 중 G동에 속한다.
타워는 탑을 가리키고 팰리스는 궁전을 가리킨다. 1999년에 착공하여 2002년에 삼성 타워팰리스 1차 완공하였고 2003년에 삼성 타워팰리스 2차 완공 이후 2004년에 완공하였고 같은 해 4월에 입주하였다. 대한민국에서 1차, 2차에 이어, 3차가 건설된 초고층 주상복합 단지이다. 완공 당시 전국에서 제일 높은 건물이 되었으며 동시에 가장 비싼 아파트였다. G동이 유일하게 3차에 해당하며, 다른 건물들과는 달리 타원형의 모습을 가지고 있다.

Page: 인천 신한은행 에스버드
Summary: 


In [67]:
chain1.invoke({"input": "하이페리온1차에 대해서 알려줘?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_wikipedia', 'arguments': '{\n"query": "하이페리온1차"\n}'}})

In [68]:
chain.invoke({"input":"신석기시대는?"})

search_wikipedia 신석기시대


"Page: 신석기 시대\nSummary: 신석기 시대(新石器時代, The Neolithic Age)는 석기 시대 후기로, 돌을 갈아 만든 간석기와 질그릇(토기)을 도구로 사용하여 식량 생산 단계에 이른 시대를 말한다. 인류사회는 구석기시대의 채집 경제로부터 신석기 시대의 생산경제로 발전하는데 이러한 생산 경제로의 전환은 인류 문화사상 하나의 전기를 가져온 사건이다. 때문에 이러한 전환을 신석기 혁명이라고도 한다. 이러한 비약을 가져온 이유의 하나로서 기후의 변화를 들 수가 있는데, 그것은 플라이스토세 빙하기(氷河期)가 끝나고 홀로세에 들어오면서 오늘날과 같은 기후로 변해 농업 생산에 적합한 시기에 돌입했기 때문에 형성된 것이다. 지금까지 알려진 가장 이른 신석기문화는 팔레스티나의 예리코(Jericho)와 이라크의 자르모(Jarmo)이다. 이들 중동의 문화는 석기 시대의 말기인 기원전 6500년 전에 시작되었으며, 홀로세의 아구석기의 시대가 끝나고 따라오는 시대이다.\n\nPage: 한국의 신석기 시대\nSummary: 한국의 신석기 시대는 한반도, 만주 일대에서 고대 한국인이 활동한 기원전 8000년부터 기원전 1500년까지의 시기를 말한다. 이를 다시 중석기와 신석기로 구분해, 기원전 2700년 이전 시기를 '중석기 시대'로 분류하기도 한다.\n뗀석기에 이어 간석기를 주로 사용하였기 때문에 '신석기'로 불리며, 대표적인 유물이 빗살무늬토기이기 때문에 빗살무늬토기 시대라고도 한다.\n강가나 바닷가에 움집을 짓고 정착 생활하였다. 다른 씨족의 혼인(족외혼을 통해 부족 사회를 형성하고, 연장자나 경험이 많은 사람이 부족을 이끄는 부족 사회였다. 신석기 혁명이라 불리는 농경이 시작되어 밭농사(조·피·수수 등 잡곡류 경작)의 중심이었지만 여전히 중요한 식량 획보 수단을 위해 어로와 사냥이 병행되었다. 토기를 만들어 음식을 조리·저장하였으며, 가락바퀴와 뼈바늘을 이용해 원시적 수공업이 생산되었다. 애니미즘, 샤머니즘, 토테미즘, 영혼 숭배, 조상 숭배 등 원시 신앙이

In [69]:
chain.invoke({"input":"태안 날씨는?"})

'The current temperature is 18.6°C'